In [ ]:
#imports packages
import json
import collections
import more_itertools
import seaborn
import pandas as pd
import random
import subprocess
import pathlib
import numpy as np
import datetime
import tqdm
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import mmap
from tqdm import tqdm
import openpyxl
import more_itertools

In [ ]:
#data -- import/upload xlsx spreadsheet
#data = pd.read_excel('/pscratch/sd/m/masare/sacct/incident_sla_AY23_complete.xlsx') #full data
data = pd.read_excel('/pscratch/sd/m/masare/sacct/incident_sla.xlsx') #small size

In [ ]:
data.info()

In [ ]:
data.head(2)

In [ ]:
data.columns

In [ ]:
#Rename Colum 'Number' to 'ID' into the dataframe
data = data.rename(columns={"Number":"ID"})
data.columns # RECHECK COLUMNS

In [ ]:
type(data)
data.dtypes

In [ ]:
data['ID']=data['ID'].astype(str) #ensuring ID column is of string type
data['ID']=data['ID'].str[3:] #removing first three characters
data['ID']=pd.to_numeric(data['ID'],errors='coerce') #converting ID column into numberic
data.head()

In [ ]:
#selecting desired columns
columns_to_keep = ['ID',
                 'Created',  
                'Title',
                'Additional comments',
                'Close notes',
                'Comments and Work notes',
                'Staff work notes (NERSC private)']
data = data.loc[:, columns_to_keep].copy()

In [ ]:
#removing spaces from column names
data.columns = data.columns.str.replace(' ', '')

In [ ]:
data.loc[:, 'Created']= pd.to_datetime(data['Created'], errors = 'coerce')

In [ ]:
print(data.dtypes)

In [ ]:
data.columns

In [ ]:
#convert columns from object to str. I have to use ".loc" to avoid the copy warning
object_columns =[ 'Title', 
                 'Additionalcomments', 
                 'Closenotes',
                 'CommentsandWorknotes', 
                 'Staffworknotes(NERSCprivate)']
for col in object_columns:
    data.loc[:, col] = data.loc[:, col].astype(str)

In [ ]:
data.head(2)

In [ ]:
data['Additionalcomments'][0]

In [ ]:
#clean 
   #isolate import columns, drop unimportant columns
   #remove time stamps (can look at my code and use that) 


text_cols = [ 'Created', 
             'Title',
            'Additionalcomments',
             'Closenotes',
            'CommentsandWorknotes',
           'Staffworknotes(NERSCprivate)'
            ]

data = data[['ID'] + text_cols].copy()
data.fillna('', inplace=True)
data = data.astype(str)

## EP -- Let's use this cell to name the columns we want our data frame. This is a little different than what I did in
# 'ticket_text_mine.py`

In [ ]:
data['Additionalcomments'][0]

In [ ]:
data.head(2)

In [ ]:
data[text_cols[5]]

In [ ]:
def process_comment_text(text):
    
    # Removed System Messages
    text = re.sub(r'No user response \d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s- System \(Additional comments\)','',text)
    text = re.sub(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s-.*?\(Additional\scomments\)','',text)
    text = re.sub(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s-.{1,60}?\(Staff work notes \(NERSC private\)\)','',text)
    
    text = re.sub(r'On\s\w{3},\s\w{3}\s\d+,\s\d{4}\sat\s\d+:\d{2}\s\w{2}','',text)
    text = re.sub(r'Incident automatically closed after \d+? days in the Resolved state.','',text)
    text = re.sub(r'Incident automatically resolved after \d+? days in waiting for user reply','',text)
                    #Incident automatically resolved after 14 days in waiting for user reply.
    
    # Remove HTML links
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
    
    #remove phonenumber
    text = re.sub(r'(\+?\d{1,2}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?\d{3}[-.\s]?\d{4}', '', text)
    
    #remove emails
    text = re.sub(r'[\w]+@[\w]+.\w{3}','',text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove specific words like 'hi', 'hello'
    unwanted_words = ['hi', 'hello','email','organization','laboratory','bye','mr','thanks','day']
    pattern = r'\b(?:{})\b'.format('|'.join(unwanted_words))
    text = re.sub(pattern, '', text)
    

    return text

In [ ]:
%%time
for col in text_cols: 
    if col not in data.columns: 
        raise ValueError(f"Column '{col}' not found in DataFrame") 


data[text_cols] = data[text_cols].fillna('').astype(str)
# Concatenate the text columns
print(data.head())
data['text_data'] = data[text_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
print(data.head())
# Apply the cleaning function
data_before = pd.DataFrame()
data_before['text_data'] = data['text_data']
data['text_data'] = data['text_data'].apply(process_comment_text)

In [ ]:
data_cleaned = data[['ID','Created','text_data']] 

In [ ]:
data_before['text_data'].iloc[0]

In [ ]:
process_comment_text(data_before['text_data'].iloc[0])

In [ ]:
data_cleaned['text_data'].iloc[0]

In [ ]:
data_cleaned.to_csv('/global/u2/m/masare/ticket-text-analysis/bert_ticket_cleaned.csv', index=False)
#data_df = pd.read_csv('/global/u2/m/masare/ticket-text-analysis/Ticket.csv')

In [ ]:
#import pandas as pd
#import spacy
#nlp = spacy.load("en_core_web_lg")
#def remove_person_names(text):
  #doc = nlp(text)
 #new_text = []
 # for token in doc:
  #  if token.ent_type_ != "PERSON":
  #    new_text.append(token.text)
   # else:
  #    new_text.append(" ")
 # return " ".join(new_text)



In [ ]:
# Apply the function to the relevant columns
#for column in ['Additionalcomments', 'Closenotes', 'CommentsandWorknotes', 'Staffworknotes(NERSCprivate)']:
 # data[column] = data[column].apply(remove_person_names)

#print(data)

In [ ]:

  
    # Removing timestamps
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}:\d{2}\s+\w{2}\b', '', text)
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}\d{2}\s+\w{2}\b', '', text, flags=re.IGNORECASE)
    #text = re.sub(r'\b(?:mon|tue|wed|thu|fri|sat|sun)\s+(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s+\d{1,2}\s+\d{4}\s+\d{1,2}[:]\d{2}\s+\w{2}\b', '', text, flags=re.IGNORECASE)
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}\d{2}\s+\w{2}\b', '', text) 
    #text = re.sub(r'\b\(?:sat|sun|mon|tue|wed|thu|fri)\.?(?:\w*day)?\s?\b', '',text)
    #mon[a-z\.]*\s+|tue[a-z\.]*\s+|wed[a-z\.]*\s+|thu[a-z\.]*\s+|fri[a-z\.]*\s+|sat[a-z\.]*\s+|sun[a-z\.]*\s+

    
    # Removing bracketed content
    #text = re.sub(r'\[.*?\]', '', text)
    #text = re.sub(r'\(.*?\)', '', text)
    # Removing extra whitespace
    #text = ' '.join(text.split())

    #Removing names
    #text = re.sub(r'\b[a-zA-Z][a-z]*\b(?:\s+[a-zA-Z][a-z]*)*', '', text) 
    #text = re.sub(r'\b[a-zA-Z-]+$\b','', text)
    #phone numbers
    #text = re.sub(r'\d\d\d-\d\d\d-\d\d\d\d','', text)
  
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+-\w+ \w+ \(Additional comments\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Additional comments\)', '', text)
    ##text  = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - System \(Additional comments\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text)
    
      #text = re.sub(r'\w+, w+ \d+, d{4} at \d{2}:\d{2} \w{2} \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)','',text)

    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Staff work notes \(NERSC private\)\)', '', text)
    #text = re.sub(r'On \w+, w+ \d+, d{4} at \d{2}:\d{2} \w{2} \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)'
            
    
    #text = re.sub(r'@\w+ \w+', '', text)
    #text = re.sub(r'\d{4}-\d{2}-\d{2}', '', text)
    # text_data = re.sub(r'\d{2}:\d{2}:\d{2}', '', text) #remove system times
    # text_data = re.sub(r'\(Additional comments\)', '', text)
    # text_data = re.sub(r'\(Staff work notes \(NERSC private\)\)', '', text)
   
    
  
    # Remove punctuations
    # text = re.sub(r'\.','', text)
    #text = re.sub(r',','', text)
    #text = re.sub(r':','', text)
    #text = re.sub(r'\?','', text)
    
   
  
        
    # removing quotes can be problematic
    # text = re.sub(r'\"','', text)
    # text = re.sub(r'\'','', text)
    #text = text.split()
    
    #wl = WordNetLemmatizer()
    #text = [wl.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    #text = ' '.join(text)

In [ ]:
#text = "2023-01-30 02:00:04 - System (Additional comments) \
#Incident automatically closed after 7 days in the Resolved state. \
#2023-01-19 07:58:19 - Yuen-Dat Chan (Additional comments) \
#reply from: ydchan@lbl.gov \
#Thank you very much! \
#On Wed, Jan 18, 2023 at 8:12 PM \
#2023-01-18 20:11:28 - Shane Canon (Additional comments) \
#This should be in place now. \
#2023-01-18 13:30:20 - Lisa Gerhardt (Additional comments) \
#Hi Yuen-Dat, \
#I've put in the request to add user chy to the majorana \
#ssh access. You should hear back soon. \
#\
#Bye, \
#Lisa"
